libraries

In [7]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

Save Models

In [8]:
saved_model_path = "Saved_Model_With_TF1"
tf.saved_model.save(model, saved_model_path)
model.save("MobileNetWithoutTheLast6Layers.h5")
print("Saved Model")
saved_model_path = "Saved_Model_With_TFWith_Keras"
tf.keras.models.save_model(model, saved_model_path)

NameError: name 'model' is not defined

Load Models

In [8]:
loaded_model = tf.saved_model.load("0_Saved_Model_With_TF1")
loaded_model_keras = tf.keras.models.load_model("0_Saved_Model_With_TFWith_Keras")
loaded_model_keras_h5 = tf.keras.models.load_model('0_MobileNetWithoutTheLast6Layers.h5', custom_objects={'KerasLayer':hub.KerasLayer})
loaded_Pruned_model_keras_h5 = tf.keras.models.load_model('MobileNetWithoutTheLast9Layers.h5', custom_objects={'KerasLayer':hub.KerasLayer})

save weights

In [5]:
loaded_model_keras_h5.save_weights("PretrainedWeights.h5")

Build dataSet from Directory

In [5]:
data_dir = 'D:/kolya/4th year/GP/New Idea/GitHub Codes/Sign-Language-Translator-main/ASL Alphabitic/asl_alphabet_train/asl_alphabet_train'

pixels = 224
IMAGE_SIZE = (pixels, pixels)
print(f"Input size {IMAGE_SIZE}")

BATCH_SIZE = 16#@param {type:"integer"}

normalization_layer = tf.keras.layers.Rescaling(1. / 255)
preprocessing_model = tf.keras.Sequential([normalization_layer])

def build_dataset(subset):
  return tf.keras.preprocessing.image_dataset_from_directory(
      data_dir,
      validation_split=.20,
      subset=subset,
      label_mode="categorical",
      # Seed needs to provided when using validation_split and shuffle = True.
      # A fixed seed is used so that the validation set is stable across runs.
      seed=123,
      image_size=IMAGE_SIZE,
      batch_size=1)

Input size (224, 224)


msh fahm hna leh b3ml kol dah 
fahm 7tt el normalization layer bs

In [14]:
normalization_layer = tf.keras.layers.Rescaling(1. / 255)
preprocessing_model = tf.keras.Sequential([normalization_layer])

train_ds = build_dataset("training")
class_names = tuple(train_ds.class_names)
train_size = train_ds.cardinality().numpy()
train_ds = train_ds.unbatch().batch(BATCH_SIZE)
train_ds = train_ds.repeat()

train_ds = train_ds.map(lambda images, labels:
                        (preprocessing_model(images), labels))

val_ds = build_dataset("validation")
valid_size = val_ds.cardinality().numpy()
val_ds = val_ds.unbatch().batch(BATCH_SIZE)
val_ds = val_ds.map(lambda images, labels:
                    (normalization_layer(images), labels))

Found 87000 files belonging to 29 classes.
Using 69600 files for training.
Found 87000 files belonging to 29 classes.
Using 17400 files for validation.


Evalute the model on the dataset

In [20]:
loaded_Pruned_model_keras_h5.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

loaded_Pruned_model_keras_h5.evaluate(val_ds,verbose = 1)

1088/1088 [==============================] - 384s 351ms/step - loss: 0.0992 - accuracy: 0.9799


[0.0992448478937149, 0.9798850417137146]

tare2a tania 3shan a3ml load l el dataset 

bst5dm m3aha .fit_generator msh .fit bs 

In [2]:
from keras.preprocessing.image import ImageDataGenerator
train_data_dir = 'D:/kolya/4th year/GP/New Idea/GitHub Codes/Sign-Language-Translator-main/ASL Alphabitic/asl_alphabet_train/asl_alphabet_train'


train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=1,
    class_mode='categorical')

Found 87000 images belonging to 29 classes.


In [6]:
import tqdm
import numpy as np
train_generator.reset()
batch_size = 1
X_train, y_train = next(train_generator)
for i in (range(int(len(train_generator)/batch_size)-1)): #1st batch is already fetched before the for loop.
  img, label = next(train_generator)
  X_train = np.append(X_train, img, axis=0 )
  y_train = np.append(y_train, label, axis=0)
print(X_train.shape, y_train.shape)

KeyboardInterrupt: 

tare2a a5od beha el DataSet bardo tania

In [2]:
import os
import cv2 
import numpy as np
path = 'D:/kolya/4th year/GP/New Idea/GitHub Codes/Sign-Language-Translator-main/ASL Alphabitic/asl_alphabet_train/asl_alphabet_train'
data = []
label = []

label_one_hot_encodded = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
labels_one_hot_encodded =[]

Files = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 
           'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 
           'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
label_val = 0
for files in Files:
    cpath = os.path.join(path, files)
    cpath = os.path.join(cpath)
    print(cpath)
    label_one_hot_encodded[label_val] = 1
    for img in os.listdir(cpath):
        image_array = cv2.imread(os.path.join(cpath, img), cv2.IMREAD_COLOR)
        image_array = cv2.resize(image_array,(224,224))
        # image_array = np.array(image_array)/255
        data.append(image_array)
        label.append(label_val)

        labels_one_hot_encodded.append(label_one_hot_encodded.copy()) 


    label_one_hot_encodded[label_val] = 0 #reseting to the old vector
    label_val = label_val + 1


from sklearn.model_selection import train_test_split
# data = np.asarray(data)
# label = np.asarray(label)

X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.33, random_state=42)

D:/kolya/4th year/GP/New Idea/GitHub Codes/Sign-Language-Translator-main/ASL Alphabitic/asl_alphabet_train/asl_alphabet_train\A
D:/kolya/4th year/GP/New Idea/GitHub Codes/Sign-Language-Translator-main/ASL Alphabitic/asl_alphabet_train/asl_alphabet_train\B
D:/kolya/4th year/GP/New Idea/GitHub Codes/Sign-Language-Translator-main/ASL Alphabitic/asl_alphabet_train/asl_alphabet_train\C


KeyboardInterrupt: 

Predicting an image

In [11]:
model = tf.keras.models.load_model('MobileNetWithoutTheLast6Layers.h5', custom_objects={'KerasLayer':hub.KerasLayer})
import cv2
classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 
           'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 
           'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']

word2 = cv2.imread("test_forC.jpeg")
word2 = cv2.resize(word2,(224,224))     # resize image to match model's expected sizing


def preprocess_Input(X):
    np_X = np.array(X)
    normalised_X = np_X.astype('float32')/255.0
    return normalised_X

word2 = preprocess_Input(word2)



prediction_scores = model.predict(np.expand_dims(word2, axis=0))
predicted_index = np.argmax(prediction_scores)
print(prediction_scores)
print("Predicted label: " + classes[predicted_index])

1/1 [==============================] - 1s 649ms/step
[[1.3639607e-10 6.9615353e-09 9.9999785e-01 2.6756791e-11 3.1700759e-10
  2.6224194e-12 3.5286693e-12 2.2860066e-08 2.1222627e-08 4.7620571e-12
  2.5463021e-12 1.0656387e-06 1.0537037e-10 2.2493651e-07 1.3452154e-08
  1.7005860e-08 5.8649471e-07 1.1057926e-12 1.7534789e-07 7.6919872e-13
  5.7855819e-15 2.1180832e-10 1.1783766e-09 1.0313684e-13 2.6977274e-09
  5.1682449e-09 2.7027306e-11 2.9993477e-10 2.5444285e-08]]
Predicted label: C


Predicting list of images

In [9]:
import tensorflow_hub as hub
import tensorflow as tf
import cv2
import os

classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 
           'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 
           'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']

path = 'Test Images'
# model = tf.keras.models.load_model('words_model_kaggle.h5', custom_objects={'KerasLayer':hub.KerasLayer})

for i in range(0,29) :
    currentImage = classes[i]+"_test.jpg"
    currentImage = os.path.join(path, currentImage)
    word2 = cv2.imread(currentImage)
    # word2 = cv2.resize(word2,(200,200))     # resize image to match model's expected sizing

    def preprocess_Input(X):
        np_X = np.array(X)
        normalised_X = np_X.astype('float32')/255.0
        return normalised_X

    word2 = preprocess_Input(word2)
    prediction_scores = loaded_Pruned_model_keras_h5.predict(np.expand_dims(word2, axis=0))
    predicted_index = np.argmax(prediction_scores)
    print("Predicted label: " + classes[predicted_index])
    print("Actual label: " + classes[i])


1/1 [==============================] - 1s 1s/step
Predicted label: A
Actual label: A
1/1 [==============================] - 0s 78ms/step
Predicted label: B
Actual label: B
1/1 [==============================] - 0s 41ms/step
Predicted label: C
Actual label: C
1/1 [==============================] - 0s 51ms/step
Predicted label: D
Actual label: D
1/1 [==============================] - 0s 61ms/step
Predicted label: E
Actual label: E
1/1 [==============================] - 0s 58ms/step
Predicted label: F
Actual label: F
1/1 [==============================] - 0s 58ms/step
Predicted label: G
Actual label: G
1/1 [==============================] - 0s 56ms/step
Predicted label: H
Actual label: H
1/1 [==============================] - 0s 52ms/step
Predicted label: I
Actual label: I
1/1 [==============================] - 0s 64ms/step
Predicted label: J
Actual label: J
1/1 [==============================] - 0s 50ms/step
Predicted label: K
Actual label: K
1/1 [==============================] - 0s 54m